In [2]:
import pandas as pd

df1 = pd.read_csv('/content/UNSW_NB15_testing-set.csv')
df2 = pd.read_csv('/content/UNSW_NB15_training-set.csv')

dataset = pd.concat([df1, df2], ignore_index=True)
print("Original dataset shape:", dataset.shape)

label_1_rows = dataset[dataset['label'] == 1]
label_0_rows = dataset[dataset['label'] == 0]

label_1_rows = label_1_rows.sample(n=93000, random_state=42)
label_0_rows = label_0_rows.sample(n=93000, random_state=42)


ds = pd.concat([label_0_rows, label_1_rows], ignore_index=True)


ds = ds.sample(frac=1, random_state=42).reset_index(drop=True)
print("Balanced and shuffled dataset shape:", ds.shape)

ds.to_csv("mixed_balanced.csv", index=False)

print(ds['label'].value_counts())


Original dataset shape: (257673, 45)
Balanced and shuffled dataset shape: (186000, 45)
label
0    93000
1    93000
Name: count, dtype: int64


In [3]:
ds.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,30044,0.306992,tcp,-,FIN,28,28,4192,2976,179.157766,...,1,3,0,0,0,10,7,0,Normal,0
1,13535,0.130261,tcp,-,FIN,20,22,4040,2664,314.752679,...,1,5,0,0,0,3,11,0,Normal,0
2,44990,0.000003,udp,-,INT,2,0,1064,0,333333.321500,...,1,2,0,0,0,1,2,0,Normal,0
3,105247,0.000006,unas,-,INT,2,0,200,0,166666.660800,...,1,1,0,0,0,2,1,0,Exploits,1
4,41978,1.146434,tcp,-,FIN,10,8,2516,354,14.828591,...,1,3,0,0,0,3,3,0,Normal,0


In [4]:
ds['service'] = ds['service'].replace({
    '-': 0,
    'dns': 1,
    'http': 2,
    'smtp': 3,
    'ftp-data': 4,
    'ftp': 5,
    'ssh': 6,
    'pop3': 7,
    'dhcp': 8,
    'snmp': 9,
    'ssl': 10,
    'irc': 11,
    'radius': 12
})

ds['state'] = ds['state'].replace({
    'FIN': 0,
    'INT': 1,
    'CON': 2,
    'REQ': 3,
    'RST': 4,
    'ECO': 5,
    'ACC': 6,
    'URN': 7,
    'no': 8,
    'PAR': 9
})

ds['attack_cat'] = ds['attack_cat'].replace({
    'Normal': 0,
    'Generic': 1,
    'Exploits': 2,
    'Fuzzers': 3,
    'DoS': 4,
    'Reconnaissance': 5,
    'Analysis': 6,
    'Backdoor': 7,
    'Shellcode': 8,
    'Worms': 9
})

/tmp/ipython-input-212674094.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds['service'] = ds['service'].replace({
/tmp/ipython-input-212674094.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds['state'] = ds['state'].replace({
/tmp/ipython-input-212674094.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downca

In [5]:
ds.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,30044,0.306992,tcp,0,0,28,28,4192,2976,179.157766,...,1,3,0,0,0,10,7,0,0,0
1,13535,0.130261,tcp,0,0,20,22,4040,2664,314.752679,...,1,5,0,0,0,3,11,0,0,0
2,44990,0.000003,udp,0,1,2,0,1064,0,333333.321500,...,1,2,0,0,0,1,2,0,0,0
3,105247,0.000006,unas,0,1,2,0,200,0,166666.660800,...,1,1,0,0,0,2,1,0,2,1
4,41978,1.146434,tcp,0,0,10,8,2516,354,14.828591,...,1,3,0,0,0,3,3,0,0,0


In [6]:
unique_proto = ds['proto'].unique()
mapping = {proto: idx for idx, proto in enumerate(unique_proto)}
ds['proto'] = ds['proto'].replace(mapping)

/tmp/ipython-input-1450688475.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds['proto'] = ds['proto'].replace(mapping)


In [7]:
x = ds.drop(['id', 'label', 'attack_cat'], axis=1)
y = ds['label']

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

x_scaled.head()


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,-0.155803,-0.245260,-0.614063,-0.864674,0.051608,0.057333,-0.021673,-0.096512,-0.503962,-1.209811,...,-0.331921,-0.468704,-0.450678,-0.421681,-0.111708,-0.111639,-0.201604,0.492917,-0.132939,-0.142032
1,-0.186869,-0.245260,-0.614063,-0.864674,-0.011710,0.003137,-0.022629,-0.098671,-0.503058,-1.209811,...,-0.331921,-0.468704,-0.450678,-0.228004,-0.111708,-0.111639,-0.201604,-0.412007,0.263259,-0.142032
2,-0.209767,-0.187028,-0.614063,0.479547,-0.154175,-0.195583,-0.041354,-0.117106,1.718234,0.856716,...,-0.599016,-0.468704,-0.450678,-0.518519,-0.111708,-0.111639,-0.201604,-0.670557,-0.628188,-0.142032
3,-0.209766,-0.128795,-0.614063,0.479547,-0.154175,-0.195583,-0.046791,-0.117106,0.606538,0.856716,...,-0.599016,-0.468704,-0.450678,-0.615357,-0.111708,-0.111639,-0.201604,-0.541282,-0.727237,-0.142032
4,-0.008243,-0.245260,-0.614063,-0.864674,-0.090857,-0.123321,-0.032218,-0.114657,-0.505059,0.856716,...,-0.599016,-0.468704,-0.450678,-0.421681,-0.111708,-0.111639,-0.201604,-0.412007,-0.529138,-0.142032


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
num_classes = len(np.unique(y))

model = Sequential([
    Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,890 (54.26 KB)

 Trainable params: 13,890 (54.26 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop]
)


Epoch 1/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8620 - loss: 0.2789 - val_accuracy: 0.9086 - val_loss: 0.1756
Epoch 2/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9088 - loss: 0.1819 - val_accuracy: 0.9185 - val_loss: 0.1656
Epoch 3/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9146 - loss: 0.1717 - val_accuracy: 0.9257 - val_loss: 0.1599
Epoch 4/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9216 - loss: 0.1634 - val_accuracy: 0.9293 - val_loss: 0.1530
Epoch 5/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9228 - loss: 0.1604 - val_accuracy: 0.9243 - val_loss: 0.1536
Epoch 6/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9243 - loss: 0.1584 - val_accuracy: 0.9234 - val_loss: 0.1524
Epoch 7/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9264 - loss: 0.1551 - val_accuracy: 0.9273 - val_loss: 0.1507
Epoch 8/50
1860/1860 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9260 - loss: 0.1557 

In [15]:
y_pred = model.predict(x_test).argmax(axis=1)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


1163/1163 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     18567
           1       0.95      0.93      0.94     18633

    accuracy                           0.94     37200
   macro avg       0.94      0.94      0.94     37200
weighted avg       0.94      0.94      0.94     37200

Confusion Matrix:
[[17660   907]
 [ 1366 17267]]


In [21]:
model.save("ids_model.h5")